In [235]:
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta

TIMELINE_IMPORTS_FILE_TEMPLATE = 'timeline_imports/{0}.kml'
TAG_PREFIX = '{http://www.opengis.net/kml/2.2}'
CYCLING_PLACEMARK_NAME = 'Cycling'
DATE_FORMAT = '%Y-%m-%dT%H:%M:%S.%fZ'

In [113]:
def find_tags(tagname, element):
    return element.findall(TAG_PREFIX+tagname)

def find_tag(tagname, element):
    return find_tags(tagname, element)[0]

def describe_children(element):
    for child in element:
        print(child.tag, child.attrib, child.keys(), child.items(), child.text)

In [114]:
# Import the timeline files
tree = ET.parse(TIMELINE_IMPORTS_FILE_TEMPLATE.format('2018-10-13'))
root = tree.getroot()

In [240]:
document = find_tag('Document', root)
placemarks = find_tags('Placemark', document)

def is_cycling(placemark):
    name = find_tag('name', placemark)
    return name.text == CYCLING_PLACEMARK_NAME

def date_parser(raw_date):
    return datetime.strptime(raw_date, DATE_FORMAT)

def formatted_step(date, step):
    return (date + timedelta(seconds=step)).strftime('%m/%d/%Y')

def create_track_points(begin_time_string, end_time_string, coords_string):
    coords = list(filter(
        lambda coords: len(coords) == 3,
        map(lambda coords: coords.split(','), coords_string)
    ))
    
    begin_time = date_parser(begin_time_string)
    
    delta_seconds = (begin_time - date_parser(end_time_string)).seconds
    step_seconds = delta_seconds/len(coords)
    
    return [{
        'lon': coord[0],
        'lat': coord[1],
        'ele': coord[2],
        'time': formatted_step(begin_time, step_seconds*index)
    } for index, coord in enumerate(coords)]
    
cycling_placemarks = filter(is_cycling, placemarks)

for placemark in cycling_placemarks:
    timeSpan = find_tag('TimeSpan', placemark)
    begin_time = find_tag('begin', timeSpan).text
    end_time = find_tag('end', timeSpan).text
    coordinates =  find_tag('coordinates', find_tag('LineString', placemark)).text.split(' ')
    
    track_points = create_track_points(begin_time, end_time, coordinates)
    
    print(track_points)

[{'lon': '13.394772699999999', 'lat': '52.493783199999996', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.394772699999999', 'lat': '52.493783199999996', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3941947', 'lat': '52.4908961', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3980648', 'lat': '52.4889648', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3981411', 'lat': '52.489003', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3982929', 'lat': '52.4890361', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3900304', 'lat': '52.4896405', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3858042', 'lat': '52.4854458', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3858454', 'lat': '52.4852005', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.38584', 'lat': '52.4848031', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3857563', 'lat': '52.4848012', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3857487', 'lat': '52.4848115', 'ele': '0', 'time': '10/13/2018'}, {'lon': '13.3853178', 'lat': '52.4

In [50]:
waypoints = find_tags('wpt')
tracks = find_tags('trk')

